In [1]:
# Import all the required libraries
import numpy as np
import pandas as pd
from datetime import datetime as dt
import itertools
%matplotlib inline

In [2]:
# Load the data from the CSV file into a dataframe using read_csv function
userPath = '/Users/prashanth/Downloads/DS Project/'
laLiga1 = pd.read_csv(userPath + '2019-20.csv')
laLiga2 = pd.read_csv(userPath + '2020-21.csv')
laLiga3 = pd.read_csv(userPath + '2021-22.csv')
laLiga4 = pd.read_csv(userPath + '2022-23.csv')
laLiga5 = pd.read_csv(userPath + '2023-24.csv')


In [3]:
from datetime import datetime as dt

def dateParsing(date):
    if date == '':
        return None
    for fmt in ('%d/%m/%Y', '%d/%m/%y'):
        try:
            return dt.strptime(date, fmt).date()
        except ValueError:
            continue
    raise ValueError(f"Date format for {date} not recognized")

# Applying the date parsing to all dataframes in this
laLiga1.Date = laLiga1.Date.apply(dateParsing)    
laLiga2.Date = laLiga2.Date.apply(dateParsing)    
laLiga3.Date = laLiga3.Date.apply(dateParsing)          
laLiga4.Date = laLiga4.Date.apply(dateParsing)    
laLiga5.Date = laLiga5.Date.apply(dateParsing)

# Extract the relevant statistics from the data
requiredColumns = ['Date','HomeTeam','AwayTeam','FTHG','FTAG','FTR']
performanceStats1 = laLiga1[requiredColumns]
performanceStats2 = laLiga2[requiredColumns]
performanceStats3 = laLiga3[requiredColumns]
performanceStats4 = laLiga4[requiredColumns]
performanceStats5 = laLiga5[requiredColumns]

# Concatonation of game playing statistics
performanceStats = pd.concat([performanceStats1, performanceStats2, performanceStats3, 
                              performanceStats4, performanceStats5])


### ***GOALS SCORED AND CONCEDED AT THE END OF EACH MATCHWEEK, ORGANIZED BY TEAM AND MATCHWEEK.***

In [4]:
 # Get the aggregated goals scored
def getGoalsScored(performanceStats):
    teams = {}
    for i in performanceStats.groupby('HomeTeam').mean().T.columns:
        teams[i] = []
    for i in range(len(performanceStats)):
        HTGS = performanceStats.iloc[i]['FTHG']
        ATGS = performanceStats.iloc[i]['FTAG']
        teams[performanceStats.iloc[i].HomeTeam].append(HTGS)
        teams[performanceStats.iloc[i].AwayTeam].append(ATGS)
   # Generate the dataframe for goals scored with teams as rows and matchweeks as columns.
    GoalsScored = pd.DataFrame(data=teams, index = [i for i in range(1,39)]).T
    GoalsScored[0] = 0
    for i in range(2,39):
        GoalsScored[i] = GoalsScored[i] + GoalsScored[i-1]
    return GoalsScored

# Get the aggregated goals scored
def getGoalsConceded(performanceStats):    
    teams = {}
    for i in performanceStats.groupby('HomeTeam').mean().T.columns:
        teams[i] = []
    for i in range(len(performanceStats)):
        ATGC = performanceStats.iloc[i]['FTHG']
        HTGC = performanceStats.iloc[i]['FTAG']
        teams[performanceStats.iloc[i].HomeTeam].append(HTGC)
        teams[performanceStats.iloc[i].AwayTeam].append(ATGC)
    GoalsConceded = pd.DataFrame(data=teams, index = [i for i in range(1,39)]).T
    GoalsConceded[0] = 0
    for i in range(2,39):
        GoalsConceded[i] = GoalsConceded[i] + GoalsConceded[i-1]
    return GoalsConceded

def getGss(performanceStats):
    GC = getGoalsConceded(performanceStats)
    GS = getGoalsScored(performanceStats)
    j = 0
    HTGS = []
    ATGS = []
    HTGC = []
    ATGC = []
    for i in range(380):
        ht = performanceStats.iloc[i].HomeTeam
        at = performanceStats.iloc[i].AwayTeam
        HTGS.append(GS.loc[ht][j])
        ATGS.append(GS.loc[at][j])
        HTGC.append(GC.loc[ht][j])
        ATGC.append(GC.loc[at][j])
        if ((i + 1)% 10) == 0:
            j = j + 1
        
    performanceStats['HTGS'] = HTGS
    performanceStats['ATGS'] = ATGS
    performanceStats['HTGC'] = HTGC
    performanceStats['ATGC'] = ATGC
    return performanceStats

# Applying it to each dataset
performanceStats1 = getGss(performanceStats1)
performanceStats2 = getGss(performanceStats2)
performanceStats3 = getGss(performanceStats3)
performanceStats4 = getGss(performanceStats4)
performanceStats5 = getGss(performanceStats5)

/var/folders/jm/xjdylqtj6r764j4f_8jvwq280000gn/T/ipykernel_61057/104211474.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  performanceStats['HTGS'] = HTGS
/var/folders/jm/xjdylqtj6r764j4f_8jvwq280000gn/T/ipykernel_61057/104211474.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  performanceStats['ATGS'] = ATGS
/var/folders/jm/xjdylqtj6r764j4f_8jvwq280000gn/T/ipykernel_61057/104211474.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

# OBTAIN CORRESPONDING POINTS:

In [5]:
def getPoints(result):
    if result == 'W':
        return 3
    elif result == 'D':
        return 1
    else:
        return 0
    

def getCumlPoints(matchres):
    matchres_points = matchres.applymap(getPoints)
    for i in range(2, 39):
        matchres_points[i] = matchres_points[i] + matchres_points[i-1]
        
    matchres_points.insert(column=0, loc=0, value=[0]*len(matchres_points))
    return matchres_points

def getMatchres(performanceStats):
    # Create a dictionary with team names as keys and assign the value 
    # corresponding to keys is a list containing the match result
    teams = {}
    for i in performanceStats.groupby('HomeTeam').mean().T.columns:
        teams[i] = []
    for i in range(len(performanceStats)):
        if performanceStats.iloc[i].FTR == 'H':
            teams[performanceStats.iloc[i].HomeTeam].append('W')
            teams[performanceStats.iloc[i].AwayTeam].append('L')
        elif performanceStats.iloc[i].FTR == 'A':
            teams[performanceStats.iloc[i].AwayTeam].append('W')
            teams[performanceStats.iloc[i].HomeTeam].append('L')
        else:
            teams[performanceStats.iloc[i].AwayTeam].append('D')
            teams[performanceStats.iloc[i].HomeTeam].append('D')
    return pd.DataFrame(data=teams, index=[i for i in range(1, 39)]).T

def getAggPoints(performanceStats):
    matchres = getMatchres(performanceStats)
    cum_pts = getCumlPoints(matchres)
    HTP = []
    ATP = []
    j = 0
    for i in range(380):
        ht = performanceStats.iloc[i].HomeTeam
        at = performanceStats.iloc[i].AwayTeam
        HTP.append(cum_pts.loc[ht][j])
        ATP.append(cum_pts.loc[at][j])

        if ((i + 1) % 10) == 0:
            j = j + 1
            
    performanceStats['HTP'] = HTP
    performanceStats['ATP'] = ATP
    return performanceStats
    
# Applying it to each dataset
performanceStats1 = getAggPoints(performanceStats1)
performanceStats2 = getAggPoints(performanceStats2)
performanceStats3 = getAggPoints(performanceStats3)
performanceStats4 = getAggPoints(performanceStats4)
performanceStats5 = getAggPoints(performanceStats5)


/var/folders/jm/xjdylqtj6r764j4f_8jvwq280000gn/T/ipykernel_61057/2267800364.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  performanceStats['HTP'] = HTP
/var/folders/jm/xjdylqtj6r764j4f_8jvwq280000gn/T/ipykernel_61057/2267800364.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  performanceStats['ATP'] = ATP
/var/folders/jm/xjdylqtj6r764j4f_8jvwq280000gn/T/ipykernel_61057/2267800364.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

# GET TEAM FORM:

In [7]:
def getForm(performanceStats,num):
    form = getMatchres(performanceStats)
    form_final = form.copy()
    for i in range(num,39):
        form_final[i] = ''
        j = 0
        while j < num:
            form_final[i] += form[i-j]
            j += 1           
    return form_final

def addForm(performanceStats,num):
    form = getForm(performanceStats,num)
    h = ['M' for i in range(num * 10)]  
    a = ['M' for i in range(num * 10)]
    j = num
    for i in range((num*10),380):
        ht = performanceStats.iloc[i].HomeTeam
        at = performanceStats.iloc[i].AwayTeam
        past = form.loc[ht][j]              
        h.append(past[num-1])                
        past = form.loc[at][j]              
        a.append(past[num-1])                   
        
        if ((i + 1)% 10) == 0:
            j = j + 1

    performanceStats['HM' + str(num)] = h                 
    performanceStats['AM' + str(num)] = a
    return performanceStats


def addFormDf(performanceStats):
    performanceStats = addForm(performanceStats,1)
    performanceStats = addForm(performanceStats,2)
    performanceStats = addForm(performanceStats,3)
    performanceStats = addForm(performanceStats,4)
    performanceStats = addForm(performanceStats,5)
    return performanceStats    
    
# Making changes to the dataframe
performanceStats1 = addFormDf(performanceStats1)
performanceStats2 = addFormDf(performanceStats2)
performanceStats3 = addFormDf(performanceStats3)
performanceStats4 = addFormDf(performanceStats4)
performanceStats5 = addFormDf(performanceStats5)

/var/folders/jm/xjdylqtj6r764j4f_8jvwq280000gn/T/ipykernel_61057/801765197.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  performanceStats['HM' + str(num)] = h
/var/folders/jm/xjdylqtj6r764j4f_8jvwq280000gn/T/ipykernel_61057/801765197.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  performanceStats['AM' + str(num)] = a
/var/folders/jm/xjdylqtj6r764j4f_8jvwq280000gn/T/ipykernel_61057/801765197.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

In [8]:
# Rearranging columns
cols = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTGS', 'ATGS', 'HTGC', 'ATGC', 'HTP', 'ATP', 'HM1', 'HM2', 'HM3',
        'HM4', 'HM5', 'AM1', 'AM2', 'AM3', 'AM4', 'AM5' ]

performanceStats1 = performanceStats1[cols]
performanceStats2 = performanceStats2[cols]
performanceStats3 = performanceStats3[cols]
performanceStats4 = performanceStats4[cols]
performanceStats5 = performanceStats5[cols]


# Get Last Year's Position as also an independent variable:

In [10]:
Standings = pd.read_csv(userPath + "Laliga_Standings.csv")
Standings.set_index(['Team'], inplace=True)
Standings = Standings.fillna(18)

def getLast(performanceStats, Standings, year):
    HomeTeamLP = []
    AwayTeamLP = []
    for i in range(380):
        ht = performanceStats.iloc[i].HomeTeam
        at = performanceStats.iloc[i].AwayTeam
        HomeTeamLP.append(Standings.loc[ht][year])
        AwayTeamLP.append(Standings.loc[at][year])
    performanceStats['HomeTeamLP'] = HomeTeamLP
    performanceStats['AwayTeamLP'] = AwayTeamLP
    return performanceStats

performanceStats1 = getLast(performanceStats1, Standings, 0)
performanceStats2 = getLast(performanceStats2, Standings, 1)
performanceStats3 = getLast(performanceStats3, Standings, 2)
performanceStats4 = getLast(performanceStats4, Standings, 3)
performanceStats5 = getLast(performanceStats5, Standings, 4)


/var/folders/jm/xjdylqtj6r764j4f_8jvwq280000gn/T/ipykernel_61057/549878816.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  performanceStats['HomeTeamLP'] = HomeTeamLP
/var/folders/jm/xjdylqtj6r764j4f_8jvwq280000gn/T/ipykernel_61057/549878816.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  performanceStats['AwayTeamLP'] = AwayTeamLP


# Get MatchWeek:

In [11]:
def getMw(performanceStats):
    j = 1
    MatchWeek = []
    for i in range(380):
        MatchWeek.append(j)
        if ((i + 1)% 10) == 0:
            j = j + 1
    performanceStats['MW'] = MatchWeek
    return performanceStats

performanceStats1 = getMw(performanceStats1)
performanceStats2 = getMw(performanceStats2)
performanceStats3 = getMw(performanceStats3)
performanceStats4 = getMw(performanceStats4)
performanceStats5 = getMw(performanceStats5)


/var/folders/jm/xjdylqtj6r764j4f_8jvwq280000gn/T/ipykernel_61057/3636330613.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  performanceStats['MW'] = MatchWeek


# FINAL DATAFRAME

In [14]:
performanceStats = pd.concat([performanceStats1,
                          performanceStats2,
                          performanceStats3,
                          performanceStats4,
                          performanceStats5,], ignore_index=True)


# Gets the form points.
def getFormPoints(string):
    sum = 0
    for letter in string:
        sum += getPoints(letter)
    return sum

performanceStats['HTFormPtsStr'] = performanceStats['HM1'] + performanceStats['HM2'] + performanceStats['HM3'] + performanceStats['HM4'] + performanceStats['HM5']
performanceStats['ATFormPtsStr'] = performanceStats['AM1'] + performanceStats['AM2'] + performanceStats['AM3'] + performanceStats['AM4'] + performanceStats['AM5']

performanceStats['HTFormPts'] = performanceStats['HTFormPtsStr'].apply(getFormPoints)
performanceStats['ATFormPts'] = performanceStats['ATFormPtsStr'].apply(getFormPoints)

# Identify Win/Loss Streaks if any.
def get_3game_ws(string):
    if string[-3:] == 'WWW':
        return 1
    else:
        return 0
    
def get_5game_ws(string):
    if string == 'WWWWW':
        return 1
    else:
        return 0
    
def get_3game_ls(string):
    if string[-3:] == 'LLL':
        return 1
    else:
        return 0
    
def get_5game_ls(string):
    if string == 'LLLLL':
        return 1
    else:
        return 0
    
performanceStats['HTWinStreak3'] = performanceStats['HTFormPtsStr'].apply(get_3game_ws)
performanceStats['HTWinStreak5'] = performanceStats['HTFormPtsStr'].apply(get_5game_ws)
performanceStats['HTLossStreak3'] = performanceStats['HTFormPtsStr'].apply(get_3game_ls)
performanceStats['HTLossStreak5'] = performanceStats['HTFormPtsStr'].apply(get_5game_ls)

performanceStats['ATWinStreak3'] = performanceStats['ATFormPtsStr'].apply(get_3game_ws)
performanceStats['ATWinStreak5'] = performanceStats['ATFormPtsStr'].apply(get_5game_ws)
performanceStats['ATLossStreak3'] = performanceStats['ATFormPtsStr'].apply(get_3game_ls)
performanceStats['ATLossStreak5'] = performanceStats['ATFormPtsStr'].apply(get_5game_ls)

performanceStats.keys()

Index(['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTGS', 'ATGS',
       'HTGC', 'ATGC', 'HTP', 'ATP', 'HM1', 'HM2', 'HM3', 'HM4', 'HM5', 'AM1',
       'AM2', 'AM3', 'AM4', 'AM5', 'HomeTeamLP', 'AwayTeamLP', 'MW',
       'HTFormPtsStr', 'ATFormPtsStr', 'HTFormPts', 'ATFormPts',
       'HTWinStreak3', 'HTWinStreak5', 'HTLossStreak3', 'HTLossStreak5',
       'ATWinStreak3', 'ATWinStreak5', 'ATLossStreak3', 'ATLossStreak5'],
      dtype='object')

In [15]:
# Calculate Goal Difference
performanceStats['HTGD'] = performanceStats['HTGS'] - performanceStats['HTGC']
performanceStats['ATGD'] = performanceStats['ATGS'] - performanceStats['ATGC']
performanceStats['DiffPts'] = performanceStats['HTP'] - performanceStats['ATP']
performanceStats['DiffFormPts'] = performanceStats['HTFormPts'] - performanceStats['ATFormPts']

# Difference in Last Year's Positions
performanceStats['DiffLP'] = performanceStats['HomeTeamLP'] - performanceStats['AwayTeamLP']
performanceStats.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTGS,ATGS,HTGC,ATGC,...,HTLossStreak5,ATWinStreak3,ATWinStreak5,ATLossStreak3,ATLossStreak5,HTGD,ATGD,DiffPts,DiffFormPts,DiffLP
0,2019-08-16,Ath Bilbao,Barcelona,1,0,H,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9.0
1,2019-08-17,Celta,Real Madrid,1,3,A,0,0,0,0,...,0,0,0,0,0,0,0,0,0,16.0
2,2019-08-17,Valencia,Sociedad,1,1,D,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3.0
3,2019-08-17,Mallorca,Eibar,2,1,H,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5.0
4,2019-08-17,Leganes,Osasuna,0,1,A,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8.0


In [18]:
# Scale the features, DiffPts , DiffFormPts, HTGD, ATGD by Matchweek.
cols = ['HTGD','ATGD','DiffPts','DiffFormPts','HTP','ATP']
performanceStats.MW = performanceStats.MW.astype(float)

for col in cols:
    performanceStats[col] = performanceStats[col] / performanceStats.MW
    def onlyHw(string):
    if string == 'H':
        return 'H'
    else:
        return 'NH'
performanceStats['FTR'] = performanceStats.FTR.apply(onlyHw)
# Testing set (2015-16 season)
performanceStatsTest = performanceStats[1520:]

In [20]:
performanceStats.to_csv(userPath + "final_dataset.csv")
performanceStatsTest.to_csv(userPath+"test.csv")